In [1]:
# import packages
import time
from datetime import timedelta, datetime, date
import arrow 
import pdb
import glob 

from os.path import join
import os
from os import listdir
from os.path import isfile, join
import csv

import numpy as np
import numpy.matlib
import pandas as pd


import io
from runeq import Config, stream

import matplotlib.pyplot as plt
from matplotlib import figure
%matplotlib inline

'''
print('Download runeq, via Terminal: pip install runeq')
Rune instructions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/00_python_installation_instructions.md
documentation Rune functions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb
Make sure that runeq is up-to-date before you run by running: 
pip install pipupgrade
pipupgrade --verbose --latest --yes

'''



"\nprint('Download runeq, via Terminal: pip install runeq')\nRune instructions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/00_python_installation_instructions.md\ndocumentation Rune functions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb\nMake sure that runeq is up-to-date before you run by running: \npip install pipupgrade\npipupgrade --verbose --latest --yes\n\n"

# Downloading RCS-Accelerometry via Rune platform

Code requires one folder with a patient-specific folder per patient.

Code requires on folder 'config files' in main path, containing the configuration files for each specific patient. Documentation about how to create these files is given on the Rune-Github: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb.

In [2]:
# set the path to the directory in which the repo exists 
path = os.path.dirname(os.getcwd())
print(path)

/Users/jeroenhabets/Desktop/UCSF_Bradykinesia_Detection/medStateDetection


In [3]:
# set up config for Rune RCS data extraction
# The Device-ID's differ from the Apple-watch device IDs!! 

# (https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb)


'''
Config files are patient and user specific. Open yaml-files to enter personal client key and access key.
To get keys: log in in Rune-labs as admin; click edit-pencil next to patients name;
go to Clients; make new Client; copy keys to yaml-file
'''

cfg_RCS02 = Config(os.path.join(path,'config_files','rune_Jeroen_watch_config_RCS02.yaml'))
cfg_RCS05 = Config(os.path.join(path,'config_files','rune_Jeroen_watch_config_RCS05.yaml'))
cfg_RCS06 = Config(os.path.join(path,'config_files','rune_Jeroen_watch_config_RCS06.yaml'))
cfg_RCS07 = Config(os.path.join(path,'config_files','rune_Jeroen_watch_config_RCS07.yaml'))

clients = {'RCS02' : stream.V1Client(cfg_RCS02),'RCS05' : stream.V1Client(cfg_RCS05),
          'RCS06': stream.V1Client(cfg_RCS06),'RCS07': stream.V1Client(cfg_RCS07)}

# device_ids are ID's for patient's RCS's, for all patient right-sided RCS
deviceIDS = {'RCS02': 'DeviceNPC700404H', 
             'RCS05': 'DeviceNPC700415H', 
             'RCS06': 'NPC700425H', 
             'RCS07': 'NPC700403H'}







In [18]:
def downloadRCSAccData(pt,y0,m0,d0,y1,m1,d1):
    '''
    Function downloads csv files with RCS accelerometry (always with gravity, no internal
    filtering) for seperate days.
    within a given time period.
    Input:
    - pt = patient as string (e.g. 'RCS02' or 'RCS05')
    - y0,m0,d0 : start date of desired timeperiod (year, month, date, e.g. 2020, 5, 1)
    - y1,m1,d1 : end date of desired timeperiod (year, month, date, e.g. 2020, 6, 1)
    
    Saves csv files in patient folder within given path-folder.
    '''
    
    '''
    Configurations for extraction via Rune platform
    Source: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb
    
    Config files are patient and user specific. Open yaml-files to enter personal client key and access key.
    To get keys: log in in Rune-labs as admin; click edit-pencil next to patients name;
    go to Clients; make new Client; copy keys to yaml-file
    '''
    
    # define timespan
    def datetime_range(start=None, end=None):
        span = end - start
        for i in range(span.days + 1):
            yield start + timedelta(days=i)
    datetimeDays = list(datetime_range(start=datetime(y0, m0, d0), end=datetime(y1, m1, d1)))
    
    # loop over all days in requested timespan
    for timeStart in datetimeDays:
        # define start time of download, as required tuple
        start = time.mktime(timeStart.timetuple()) #1577865600, is equivalent to: 01/01/2020 @ 12:00am (UTC)
        # download per day, so end of download is end of day
        timeEnd = timeStart + timedelta(days=1)
        # convert end-time in required time tuple        
        end = time.mktime(timeEnd.timetuple()) # convert to unix time

        # set download parameters with set start and endtimes
        downloadParameters = {'device_id': deviceIDS[pt], 'start_time': start,'end_time': end, 'timestamp': 'unix'}

        '''
        Rune-function which makes API call for accel data, saves to CSV and outputs dataframe.
        RCS only generates acc-data with gravity. So, use expression=accel.
        '''
        # set download parameters with specific device, start and end date
        accessor = clients[pt].Accel(**downloadParameters, expression= "accel" )
        # Extraction of Rune function
        df = pd.DataFrame()
        for page in accessor.iter_csv_text():
            page_df = pd.read_csv(io.StringIO(page))
            df = df.append(page_df, ignore_index=True)
              
        # define name of day-file
        day = timeStart.strftime("%d") # generate 2-digit day code
        month = timeStart.strftime("%m") # generate 3-letter month code
        year = timeStart.strftime("%Y") # generate 4-digit year code

        if df.shape[0] < 5: # selects days without available acc data
            print('%s %s %s had no recording.'% ( year, month, day)) # informing about days without recording
            continue # goes further to next day without writing file
        
        fileName = '%s_%s%s%s_RCS_accel.csv' % (pt, year, month, day)
        # save dataframe to a csv
        patient_dir_name = os.path.join(path,'data','%s' %pt)
        try: 
            os.makedirs(patient_dir_name)
        except OSError as error: 
            print(error)
        df.to_csv(os.path.join(patient_dir_name,fileName), index=False)
        print('%s %s %s is written!'% ( year, month, day))
    
    return df



In [19]:
test = downloadRCSAccData(pt='RCS02', y0=2020, m0=6, d0=10, y1=2020, m1=6, d1=11)



[Errno 17] File exists: '/Users/jeroenhabets/Desktop/UCSF_Bradykinesia_Detection/medStateDetection/data/RCS02'
2020 06 10 is written!
2020 06 11 had no recording.
